In [22]:
import pandas as pd
import glob
import os

In [23]:
current_path = os.getcwd()
parent_directory = os.path.dirname(current_path)
source_path = os.path.join(parent_directory, 'data', 'raw')
destination_path =os.path.join(parent_directory, 'data', 'processed', 'combine.xlsx')

file = glob.glob(os.path.join(source_path, "*.xlsx"))

df = pd.read_excel(file[0], index_col=None)
raw_shape = df.shape

In [24]:
df = df.sort_values(['Numer osobowy','Rodz. nieob./obecn.', 'Data od'])

df['Nowa_data_zakonczenia'] = df['Data do']
df['Dokonano_złączenia'] = 0

df.drop('Zm. dnia', axis=1, inplace=True)

In [25]:
def combine_lines(df):
    changes = 0
    for i in range(0, len(df)-1):
        if (i in df.index and 
            (i+1) in df.index and
            df.loc[i, 'Numer osobowy'] == df.loc[i+1, 'Numer osobowy'] and
            df.loc[i, 'Rodz. nieob./obecn.'] == df.loc[i+1, 'Rodz. nieob./obecn.'] and
            df.loc[i, 'Data do'] == df.loc[i+1, 'Data od'] - pd.Timedelta(days=1)):

            df.loc[i, 'Nowa_data_zakonczenia'] = df.loc[i+1, 'Data do']

            df.loc[i, 'Data do'] = df.loc[i+1, 'Data do']
            df.loc[i, 'Godziny nieobecności'] += df.loc[i+1, 'Godziny nieobecności']
            df.loc[i, 'Dni obecn./nieobecn.'] += df.loc[i+1, 'Dni obecn./nieobecn.']
            df.loc[i, 'Dni kalendarzowe'] += df.loc[i+1, 'Dni kalendarzowe']
            df.loc[i, 'Dokonano_złączenia'] = 'Tak'
            df.drop(index=[i+1], inplace=True)
            changes += 1
    
    return df, changes

In [26]:
total_changes = 0
while True:
    new_df, changes = combine_lines(df)
    
    if changes == 0:
        break

    total_changes += changes
    df = new_df
    df.reset_index(drop=True, inplace=True)

print(f'Combine rows - from {raw_shape[0]} to {df.shape[0]} rows.')
display(df)

Combine rows - from 16307 to 12717 rows.


,Numer osobowy,Nazwisko pracownika lub kandydata,Rodz. nieob./obecn.,Nieobecność,Data od,Data do,Godziny nieobecności,Dni obecn./nieobecn.,Dni kalendarzowe,Autor zmiany,Administrator danych podstawow,Nowa_data_zakonczenia,Dokonano_złączenia
0,100076,Białecka Bożena,NZ,Praca / do zapłaty,2023-02-10,2023-02-10,4.0,0.0,0,KUBIAKA,Renata Adamek,2023-02-10,0
1,100076,Białecka Bożena,NZ,Praca / do zapłaty,2023-02-12,2023-02-12,4.0,0.0,0,KUBIAKA,Renata Adamek,2023-02-12,0
2,100076,Białecka Bożena,NZ,Praca / do zapłaty,2023-02-14,2023-02-14,4.0,0.0,0,KUBIAKA,Renata Adamek,2023-02-14,0
3,100076,Białecka Bożena,NZ,Praca / do zapłaty,2023-02-16,2023-02-16,4.0,0.0,0,KUBIAKA,Renata Adamek,2023-02-16,0
4,100076,Białecka Bożena,NZ,Praca / do zapłaty,2023-02-18,2023-02-18,4.0,0.0,0,KUBIAKA,Renata Adamek,2023-02-18,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12712,136719,Podstawska Greta,DWPN,Dzień wolny za pr. w nd,2023-12-06,2023-12-06,0.0,0.0,1,CZARNEC1,Renata Adamek,2023-12-06,0
12713,136719,Podstawska Greta,DWPN,Dzień wolny za pr. w nd,2023-12-12,2023-12-12,0.0,0.0,1,CZARNEC1,Renata Adamek,2023-12-12,0
12714,136719,Podstawska Greta,DWPN,Dzień wolny za pr. w nd,2023-12-14,2023-12-14,0.0,0.0,1,CZARNEC1,Renata Adamek,2023-12-14,0
12715,136719,Podstawska Greta,DWPŚ,Dzień wolny za pr. w św.,2023-12-20,2023-12-20,0.0,0.0,1,CZARNEC1,Renata Adamek,2023-12-20,0


In [27]:
df.to_excel(destination_path, index=False)